1. รหัส ชื่อ และหมู่ของนิสิตในกลุ่ม
6710451208 มนัสนันท์ รัตนเกียรติขจร 200
6710451402 สิรินดา ศิลปเจริญ 200

2. วัตถุประสงค์ของระบบต้นแบบ
เพื่อพัฒนาพัฒนาระบบช่วยวิเคราะห์และคาดการณ์ความเสี่ยงในการลาออกของพนักงาน (Employee Attrition) 
ใช้เทคนิค Feature Engineering เพื่อปรับปรุงคุณภาพข้อมูล 
โดยที่ถ้าพนักงานลาออก (Attrition) จะแสดงเป็น 1 และถ้าไม่ลาออกจะแสดงเป็น 0

3. ลิงค์ไปยังข้อมูลที่จะใช้ในระบบต้นแบบ

In [2]:
import pandas as pd
df_employee = pd.read_csv("WA_Fn-UseC_-HR-Employee-Attrition.csv")
df_employee

,Age,Attrition,BusinessTravel,DailyRate,Department,DistanceFromHome,Education,EducationField,EmployeeCount,EmployeeNumber,...,RelationshipSatisfaction,StandardHours,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager
0,41,Yes,Travel_Rarely,1102,Sales,1,2,Life Sciences,1,1,...,1,80,0,8,0,1,6,4,0,5
1,49,No,Travel_Frequently,279,Research & Development,8,1,Life Sciences,1,2,...,4,80,1,10,3,3,10,7,1,7
2,37,Yes,Travel_Rarely,1373,Research & Development,2,2,Other,1,4,...,2,80,0,7,3,3,0,0,0,0
3,33,No,Travel_Frequently,1392,Research & Development,3,4,Life Sciences,1,5,...,3,80,0,8,3,3,8,7,3,0
4,27,No,Travel_Rarely,591,Research & Development,2,1,Medical,1,7,...,4,80,1,6,3,3,2,2,2,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1465,36,No,Travel_Frequently,884,Research & Development,23,2,Medical,1,2061,...,3,80,1,17,3,3,5,2,0,3
1466,39,No,Travel_Rarely,613,Research & Development,6,1,Medical,1,2062,...,1,80,1,9,5,3,7,7,1,7
1467,27,No,Travel_Rarely,155,Research & Development,4,3,Life Sciences,1,2064,...,2,80,1,6,0,3,6,2,0,3
1468,49,No,Travel_Frequently,1023,Sales,2,3,Medical,1,2065,...,4,80,0,17,3,2,9,6,0,8


4. ปั​ญห​า ML ของตั​วเอ​งเป็นปั​ญหา​ป​ระเภ​ทใด 
- Classification

5. f​eat​ur​es ที่ใช้ในการทำนา​ยมีอะ​ไรบ้าง
- อายุ (Age)
- ระยะเวลาทำงาน (YearsAtCompany)
- ระดับความพึงพอใจ (JobSatisfaction)
- จำนวนชั่วโมงทำงาน (OverTime)
- ระดับการศึกษา (Education)
- รายได้ (MonthlyIncome)
- การเลื่อนตำแหน่ง (YearsSinceLastPromotion)


6. เฉ​ลย (label) ที่ต้​องกา​รทำนา​ยคื​ออ​ะไร
 - Attrition (สถานะของพนักงานว่า ลาออก/ไม่ลาออก) โดยทั่วไป
 ลาอออก = 1
 ไม่ลาออก = 0

7. ระบบของตัวเองสาม​ารถใช้ al​gorith​m อะไรได้บ้าง เพ​ราะเห​ตุใด

- Logistic Regression 
เข้าใจง่ายเหมาะกับการนำเสนอให้ผู้บริหารหรือ HR ตีความได้ตรงไปตรงมา เร็วและเบาใช้ทรัพยากรน้อย เหมาะกับการ deploy เบื้องต้น

- Random Forest
แม่นยำสูง ใช้การรวมหลาย Decision Trees เพื่อลดความผิดพลาด จัดการกับข้อมูลไม่สมดุลได้ดี วิเคราะห์ feature สำคัญได้ ช่วยให้ HR เข้าใจว่าอะไรส่งผลต่อการลาออก ทนต่อ noise และเหมาะกับข้อมูลจริงที่มีความซับซ้อน

In [1]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score

In [4]:
X = df_employee.drop("Attrition", axis=1)
y = df_employee["Attrition"].map({"Yes":1, "No":0})
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

num_cols = X.select_dtypes(include=["int64","float64"]).columns
cat_cols = X.select_dtypes(include=["object"]).columns

preprocessor = ColumnTransformer([
    ("num", StandardScaler(), num_cols),
    ("cat", OneHotEncoder(handle_unknown="ignore"), cat_cols)
])

logreg_pipeline = Pipeline([
    ("preprocessor", preprocessor),
    ("classifier", LogisticRegression(max_iter=1000, random_state=42))
])

rf_pipeline = Pipeline([
    ("preprocessor", preprocessor),
    ("classifier", RandomForestClassifier(n_estimators=300, random_state=42))
])

for name, pipeline in [("Logistic Regression", logreg_pipeline), ("Random Forest", rf_pipeline)]:
    print(f"\n=== {name} ===")
    pipeline.fit(X_train, y_train)
    y_pred = pipeline.predict(X_test)

    print("Accuracy:", accuracy_score(y_test, y_pred))
    print(classification_report(y_test, y_pred, digits=4))


=== Logistic Regression ===
Accuracy: 0.8605442176870748
              precision    recall  f1-score   support

           0     0.8843    0.9595    0.9204       247
           1     0.6154    0.3404    0.4384        47

    accuracy                         0.8605       294
   macro avg     0.7499    0.6500    0.6794       294
weighted avg     0.8413    0.8605    0.8433       294


=== Random Forest ===
Accuracy: 0.8469387755102041
              precision    recall  f1-score   support

           0     0.8556    0.9838    0.9153       247
           1     0.6000    0.1277    0.2105        47

    accuracy                         0.8469       294
   macro avg     0.7278    0.5557    0.5629       294
weighted avg     0.8148    0.8469    0.8026       294



8. แต่ละ algor​ithm มี hype​rpar​am​eter อะไ​รบ้าง ปรับเป็​นอะไรไ​ด้บ้าง
1) Logistic Regression 
    -penalty ช่วยลดการ overfit กับฟีเจอร์ที่มีมาก เช่น Age, JobRole, MonthlyIncome 
        ค่าเริ่มต้นในsklearn l2
        สามารถปรับเป็น l1, l2, elasticnet, none
    -C ปรับเพื่อควบคุมความซับซ้อนโมเดลให้เหมาะสมกับข้อมูล
        ค่าเริ่มต้นในsklearn 1.0
        สามารถปรับเป็น float ex. 0.01, 0.1, 1, 10, 100
    -solver วิธีการหาคำตอบของ logistic regression
        ค่าเริ่มต้นในsklearn lbfgs
        สามารถปรับเป็น liblinear, saga, newton-cg, lbfgs
    -max_iter จำนวนรอบการฝึกโมเดลสูงสุด
        ค่าเริ่มต้นในsklearn 100
        สามารถปรับเป็น 200, 500, 1000
    -class_weight การถ่วงน้ำหนักคลาสต่าง ๆ (เพื่อแก้ class imbalance)
        ค่าเริ่มต้นในsklearn None
        สามารถปรับเป็น balanced หรือ dictionary เช่น {0:1,1:5}
    -fit

2) Random Forest
    -n_estimators จำนวนต้นไม้ในป่า (จำนวน Decision Trees)
        ค่าเริ่มต้นในsklearn 100
        สามารถปรับเป็น 200, 500, 1000
    -max_depth ความลึกสูงสุดของแต่ละต้นไม้
        ค่าเริ่มต้นในsklearn None
        สามารถปรับเป็น 10, 20, 30
    -min_samples_split จำนวนข้อมูลขั้นต่ำที่ต้องมีในโหนดเพื่อแยกต่อ
        ค่าเริ่มต้นในsklearn 2
        สามารถปรับเป็น 5, 10
    -min_samples_leaf จำนวนตัวอย่างขั้นต่ำที่ต้องมีในโหนดใบไม้
        ค่าเริ่มต้นในsklearn 1
        สามารถปรับเป็น 2, 4
    -max_features จำนวนฟีเจอร์ที่สุ่มเลือกในแต่ละ split
        ค่าเริ่มต้นในsklearn auto or sqrt
        สามารถปรับเป็น log2, จำนวน
    -class_weight ถ่วงน้ำหนักคลาสต่าง ๆ
        ค่าเริ่มต้นในsklearn None
        สามารถปรับเป็น balanced 
    -bootstrap ใช้วิธี bootstrap sample ในการสร้างแต่ละต้นไม้
        ค่าเริ่มต้นในsklearn True
        สามารถปรับเป็น False

9. คำตอบที่ได้จากการทำนายของ model เบื้องต้นปกติหรือไม่ อย่างไร ต้องมีการ p​os​t-p​roc​ess หรือไม่ อย่างไร

คำตอบที่ได้จากการทำนายของ model เบื้องต้นปกติแต่ยังไม่พร้อมใช้งานทันที โดยเฉพาะกับข้อมูลลาออกของพนักงาน ซึ่งมักมี class imbalance (ลาออกน้อยกว่ามาก) ผลลัพธ์อาจปกติดีแต่ไม่แม่นยำ จึงต้องมีการทำ post-process เพื่อผลลัพธ์มีความแม่นยำมากขึ้นและนำไปใช้ได้จิง เช่น การทำ Feature Importance Analysis คือการดูว่า feature ไหนมีผลต่อโมเดลมากที่สุด, Class Balancing เพื่อให้โมเดลไม่ bias กับกลุ่มไม่ลาออก


10. ปร​ะเด็น​อื่น ๆ ที่เกี่ยว​ข้องกั​บข้อมูลของ​ตัวเอง
- Feature Correlation
บาง features อาจมีความสัมพันธ์กันสูง เช่น YearsAtCompany กับ TotalWorkingYears
ควรตรวจสอบด้วย correlation matrix และพิจารณา feature selection เพื่อลด multicollinearity

11. ก​าร​มีส่วน​ร่วมของสม​าชิกแต่ละ​คนในกลุ่ม (แต่ล​ะคนทำอะไรบ้าง)
6710451208 มนัสนันท์ รัตนเกียรติขจร 200
- ตอบคำถามและหาข้อมูลเกี่ยวกับข้อ 3,5,7,9
- ช่วยกันออกความเห็นในการตอบคำถามข้ออื่น ๆ 

6710451402 สิรินดา ศิลปเจริญ 200
- ตอบคำถามและหาข้อมูลเกี่ยวกับ 4,6,8,10
- ช่วยกันออกความเห็นในการตอบคำถามข้ออื่น ๆ 

12. กา​รเปิ​ดเผ​ยกา​รใช้เค​รื่อ​งมือปัญ​ญาป​ระดิษฐ์ (ใช้อะไร ใช้เพื่ออะไร ใช้อย่างไร, pro​mpt อย่างไร)
- ช่วยเรียบเรียงคำพูดให้สละสลวย(มีการส่งข้อมูลที่เขียนขึ้นเองเข้าไปให้เรียบเรียงคำพูด)
- ช่วยแนะนำ Algorithm ต่างๆ
- ให้คำแนะนำและอธิบายแนวคิด